In [ ]:
# !pip install langchain langchain_huggingface langchain_chroma chromadb sentence-transformers


### Chat Models

#### Without Memory

In [7]:
from langchain_aws import ChatBedrockConverse
import json

In [8]:
model_id = "amazon.nova-lite-v1:0"
llm = ChatBedrockConverse(
    model = model_id
)

In [11]:
messages = [
    {
        "role": "user",
        "content": [{"text": "Hi, I am Saqlain."}]
    }
]

body = json.dumps({
    "schemaVersion": "messages-v1",
    "messages": messages,
    "inferenceConfig": {
        "maxTokens": 500,
        "temperature": 0.7,
        "topP": 0.9
    }
})



In [12]:
response = llm.invoke(body)

In [13]:
print(response.content)

Hello Saqlain! It's nice to meet you. How can I assist you today? If you have any questions or need help with something, feel free to let me know.


In [14]:
messages = [
    {
        "role": "user",
        "content": [{"text": "Can you tell my name?"}]
    }
]

body = json.dumps({
    "schemaVersion": "messages-v1",
    "messages": messages,
    "inferenceConfig": {
        "maxTokens": 500,
        "temperature": 0.7,
        "topP": 0.9
    }
})



In [15]:
response = llm.invoke(body)

In [16]:
print(response.content)

I'm sorry, but I can't tell you your name as it's personal information that I can't access or share. If you have any other questions or need assistance with something else, feel free to ask!


#### With Memory

In [30]:
from langchain_core.messages import HumanMessage, AIMessage


chat_history = []

user_input_1 = HumanMessage(content="Hi, I am Saqlain.")
chat_history.append(user_input_1)

response_1 = llm.invoke(chat_history)
print("AI:", response_1.content)
chat_history.append(response_1)



AI: Hello Saqlain! It's nice to meet you. How can I assist you today? If you have any questions or need information on a particular topic, feel free to let me know.


In [31]:
user_input_2 = HumanMessage(content="What is my name?")
chat_history.append(user_input_2)

response_2 = llm.invoke(chat_history)
print("AI:", response_2.content)
chat_history.append(response_2)

AI: Your name is Saqlain. If you have any questions or need further assistance, feel free to ask!


### Messages

In [27]:
import json
from langchain_aws import ChatBedrockConverse
from langchain_core.messages import HumanMessage

# Initialize the model
model_id = "amazon.nova-lite-v1:0"
llm = ChatBedrockConverse(model=model_id)

# --- 1. Using a plain string input ---
plain_input = "Hi, I am Saqlain."
response_plain = llm.invoke(plain_input)
print("Response to plain input:")
print(response_plain.content)

# --- 2. Using structured messages ---
structured_message = HumanMessage(content="Hi, I am Saqlain.")
response_structured = llm.invoke([structured_message])
print("\nResponse to structured message:")
print(response_structured.content)


Response to plain input:
Hello Saqlain! It's nice to meet you. If there's anything you'd like to discuss or any questions you have, feel free to let me know. Whether it's about a specific topic, a problem you're facing, or just a general inquiry, I'm here to help!

Response to structured message:
Hello Saqlain! It's nice to meet you. How can I assist you today? If you have any questions or need information on a particular topic, feel free to let me know.


In [28]:
print(structured_message)

content='Hi, I am Saqlain.' additional_kwargs={} response_metadata={}


### Few Shot Prompting

#### Without Few Shot

In [46]:
from langchain_aws import ChatBedrockConverse
from langchain_core.messages import HumanMessage

# Initialize the chat model
model_id = "amazon.nova-lite-v1:0"
llm = ChatBedrockConverse(model=model_id)

# User input
user_input = "Who is President of Pakistan?"

# Generate response
response = llm.invoke([HumanMessage(content=user_input)])

# Output the response
print(response.content)


As of my most recent update, the President of Pakistan is Arif Alvi. However, it's important to note that political positions can change, and for the most current information, you should consult reliable and up-to-date sources such as official government websites or reputable news outlets.


In [90]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma


# Your in-memory data
texts = [
    "The capital of France is Paris.",
    "The largest planet in our solar system is Jupiter.",
    "The Great Wall of China is visible from space.",
    "President of Pakistan is Asif Ali Zardari."
]

# Optional metadata for each document
metadatas = [
    {"source": "fact1"},
    {"source": "fact2"},
    {"source": "fact3"},
    {"source": "fact4"}
]

# Initialize the embedding model
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model)

# Initialize Chroma vector store with in-memory data
vectorstore = Chroma.from_texts(
    texts=texts,
    embedding=embeddings,
    metadatas=metadatas,
    persist_directory="./vectorstore"  # Optional: specify if you want persistence
)


In [101]:
# User's query
query = "Who is President of Pakistan?"

# Retrieve top 2 relevant documents
results = vectorstore.similarity_search(query, k=2)

# Extract the content of the retrieved documents
retrieved_chunks = [doc.page_content for doc in results]


In [102]:
retrieved_chunks

['Question: Who is president of Pakistan?\nAnswer:',
 'President of Pakistan is Asif Ali Zardari.']

In [105]:
from langchain_aws import ChatBedrockConverse
from langchain_core.messages import HumanMessage

# Initialize the chat model
model_id = "amazon.nova-lite-v1:0"
llm = ChatBedrockConverse(model=model_id)

# Construct the prompt with retrieved context
context = "\n".join(retrieved_chunks)
prompt = f"Answer only based on given context below Context:\n{context}\n\nQuestion: {query}\nAnswer:"

print(prompt)


Answr only based on given context below Context:
Question: Who is president of Pakistan?
Answer:
President of Pakistan is Asif Ali Zardari.

Question: Who is President of Pakistan?
Answer:


In [106]:
response = llm.invoke([HumanMessage(content=prompt)])

# Output the response
print(response.content)

President of Pakistan is Asif Ali Zardari.

(Note: The context provided only includes information about Asif Ali Zardari being the President of Pakistan. If the context changes or if new information is available, the answer might need to be updated accordingly.)


### Structured Output

#### Without Structured Output

In [137]:
from langchain_aws import ChatBedrockConverse
from langchain_core.messages import HumanMessage
import json

# Initialize the chat model
llm = ChatBedrockConverse(model="amazon.nova-lite-v1:0")

# Craft a prompt that instructs the model to respond in JSON format
prompt = (
    "Tell me a joke about cats. "
    "Respond in JSON format with the following fields:\n"
    "{\n"
    '  "setup": "string",\n'
    '  "punchline": "string"\n'
    "}"
)

# Invoke the model with the crafted prompt
response = llm.invoke([HumanMessage(content=prompt)])

# Extract and parse the content
content = response.content
joke = json.loads(content)


#### With Structured Output

In [121]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser


class Joke(BaseModel):
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline of the joke")



In [141]:
prompt = (
    "Tell me a joke about {topic}. "
    "Respond in JSON format with the following fields:\n"
    "{{\n"
    '  "setup": "string",\n'
    '  "punchline": "string"\n'
    "}}"
)


In [142]:
from langchain_aws import ChatBedrockConverse
from langchain_core.messages import HumanMessage
import json

# Initialize the chat model
llm = ChatBedrockConverse(model="amazon.nova-lite-v1:0")

# Invoke the model
model_input = prompt.format(topic = 'cats') 
response = llm.invoke([HumanMessage(content=model_input)])

# Extract and parse the content



In [143]:
content = response.content
parsed_output = Joke(**json.loads(content))
print(parsed_output)

setup='Why did the cat join the Red Cross?' punchline='Because it wanted to be a first-aid kit!'


#### Langchain Expression Language (LCEL)

In [150]:
from langchain_core.prompts import PromptTemplate
from langchain_aws.chat_models import ChatBedrockConverse
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

# Define the Pydantic model for structured output
class Joke(BaseModel):
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline of the joke")

# Initialize the output parser with the Pydantic model
parser = JsonOutputParser(pydantic_object=Joke)

# Create the prompt template with format instructions
prompt = PromptTemplate(
    template="Tell me a joke about {topic}.\nRespond in JSON format with the following fields:\n{{\n 'setup': 'string',\n 'punchline': 'string'\n}}",
    input_variables=["topic"],
)

# Initialize the chat model
model = ChatBedrockConverse(model="amazon.nova-lite-v1:0", temperature=0)

# Chain the components using the pipe operator
chain = prompt | model | parser

# Invoke the chain to get the structured joke
joke = chain.invoke({"topic": "cats"})

# Access and print the structured fields
print("Setup:", joke['setup'])
print("Punchline:", joke['punchline'])


Setup: Why did the cat join the Red Cross?
Punchline: Because she wanted to be a first-paw responder!


### Integrating Bedrock Knowledge Bases 

- Create a separate User
- Create an S3 bucket
- Add PDF file in it
- Create Knowledge base in Bedrock with default settings

In [192]:
from langchain_aws.retrievers import AmazonKnowledgeBasesRetriever

retriever = AmazonKnowledgeBasesRetriever(
    knowledge_base_id="G8GBOPS6ZT",
    retrieval_config={"vectorSearchConfiguration": {"numberOfResults": 2}},
)

In [198]:
query = "tell me about K2 mountain and Pakistan"

In [200]:
response = retriever.invoke(query)

In [204]:
response[0].page_content

"It has the nakedness of the world before the first man—or of the cindered planet after the last.[29]     André Weil named K3 surfaces in mathematics partly after the beauty of the mountain K2.[30]     K2 lies in the northwestern Karakoram Range. It is located in the Baltistan region of Gilgit–Baltistan, Pakistan, and the Taxkorgan Tajik Autonomous County of Xinjiang, China.[a] The Tarim sedimentary basin borders the range on the north and the Lesser Himalayas on the south. Melt waters from glaciers, such as those south and east of K2, feed agriculture in the valleys and contribute significantly to the regional fresh-water supply.     K2 is ranked 22nd by topographic prominence, a measure of a mountain's independent stature. It is a part of the same extended area of uplift (including the Karakoram, the Tibetan Plateau, and the Himalayas) as Mount Everest, and it is possible to follow a path from K2 to Everest that goes no lower than 4,594 metres (15,072 ft), at the Kora La on the Nepal

In [213]:
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import BaseMessage
from langgraph.graph import StateGraph, END
from typing import Annotated, Sequence, TypedDict
from langgraph.graph.message import add_messages


In [251]:
@tool
def add_numbers(i : int, j, int) -> int:
    """Adds numbers provided in the query."""
    return i + j


@tool
def knowledge_retriever(query: str) -> str:
    """Retrieves information based on the query from Knowledge base related to Mountains in Pakistan."""
    knowledge = ""
    for response in retriever.invoke(query):
        knowledge += response.page_content + "\n"
    return knowledge

In [252]:
tools = [add_numbers, knowledge_retriever]

agent_node = create_react_agent(llm, tools)

In [253]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    
graph = StateGraph(AgentState)
graph.add_node("agent", agent_node)
graph.set_entry_point("agent")
graph.set_finish_point("agent")

# Compile the graph
app = graph.compile()

In [254]:
from langchain_core.messages import HumanMessage

response = app.invoke({"messages": [HumanMessage(content="What is 45 plus 60?")]})
print(response["messages"][-1].content)


<thinking>The error persists despite multiple attempts to call the 'add_numbers' tool correctly. The tool's validation seems to be expecting an 'int' parameter, which is not required based on the tool's description. Given the persistent error, I will inform the User about the issue and suggest they check the tool's documentation or contact support for further assistance.</thinking>

I'm sorry, but there seems to be an issue with the 'add_numbers' tool. It's expecting an 'int' parameter, which is not required based on the tool's description. I recommend checking the tool's documentation or contacting support for further assistance. If you have any other questions or need assistance with a different tool, feel free to ask.


In [255]:

response = app.invoke({"messages": [HumanMessage(content="What is the height f K2 Mountain?")]})
print(response["messages"][-1].content)

<thinking>The knowledge_retriever tool has provided the information about the height of K2 Mountain. I can now directly answer the User's question.</thinking>

The height of K2 Mountain is 8,611 meters (28,251 feet). It is the second-highest mountain on Earth, after Mount Everest.
